In [46]:
import pandas as pd
import numpy as np
import json
from shapely.geometry import shape, Point
import collections

In [38]:
neighborhoodfile = r"../geodata/Analysis Neighborhoods.geojson"
with open(neighborhoodfile) as f:
    neighborhoods_js = json.load(f)

In [49]:
# Create dict of neighborhoods
neighborhoods = collections.OrderedDict()
nhood_robbery_counts = collections.OrderedDict()
for feature in neighborhoods_js['features']:
    neighborhoods[feature['properties']['nhood']] = shape(feature['geometry'])
    nhood_robbery_counts[feature['properties']['nhood']] = 0
len(neighborhoods)

41

In [40]:
# Get robberies data
data = pd.read_csv('../rawdata/SFPD_Incidents_-_from_1_January_2003.csv')
robbery_street = data[(data.Category == 'ROBBERY') & data.Descript.str.contains("ON THE STREET") ]
print robbery_street.shape
robbery_street.head(1)

(25070, 13)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
18,160213527,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Sunday,03/13/2016,22:40,MISSION,NONE,3400 Block of CESAR CHAVEZ ST,-122.419269,37.748032,"(37.7480322172379, -122.419268840041)",16021352703014


In [51]:
nhoodlist = []

failedcount = 0
for row in robbery_street.itertuples(index=False):
    
    point = Point(getattr(row,'X'),getattr(row,'Y'))
    
    found = False
    for name, polygon in neighborhoods.iteritems():
        
        if polygon.contains(point):
            nhoodlist.append(name)
            nhood_robbery_counts[name] += 1
            found = True
    if not found:
        failedcount +=1
        nhoodlist.append(None)

print 'len(nhoodlist)', len(nhoodlist)
print 'failed', failedcount

len(nhoodlist) 25070
failed 31


In [53]:
nhood_robbery_counts

OrderedDict([(u'Bayview Hunters Point', 1710),
             (u'Bernal Heights', 712),
             (u'Castro/Upper Market', 697),
             (u'Chinatown', 418),
             (u'Excelsior', 918),
             (u'Financial District', 1184),
             (u'Glen Park', 138),
             (u'Golden Gate Park', 196),
             (u'Haight Ashbury', 354),
             (u'Hayes Valley', 707),
             (u'Inner Richmond', 178),
             (u'Inner Sunset', 145),
             (u'Japantown', 137),
             (u'Lakeshore', 157),
             (u'Lincoln Park', 1),
             (u'Lone Mountain/USF', 211),
             (u'Marina', 300),
             (u'McLaren Park', 59),
             (u'Mission', 3418),
             (u'Mission Bay', 79),
             (u'Nob Hill', 563),
             (u'Noe Valley', 184),
             (u'North Beach', 400),
             (u'Oceanview/Merced/Ingleside', 526),
             (u'Outer Mission', 720),
             (u'Outer Richmond', 268),
             (u'Pac

In [54]:
for feature in neighborhoods_js['features']:
    feature['properties']['robcount'] = nhood_robbery_counts[feature['properties']['nhood']]

In [56]:
neighborhoods_js['features'][0]

{u'geometry': {u'coordinates': [[[[-122.36691209230094, 37.732231774038944],
     [-122.36691408515034, 37.73222827449881],
     [-122.36691252579077, 37.73223130255256],
     [-122.36691209230094, 37.732231774038944]]],
   [[[-122.38122156621652, 37.7529180775117],
     [-122.38121404054337, 37.752918109322756],
     [-122.38120651524687, 37.752918245668006],
     [-122.38119899597736, 37.752918485556044],
     [-122.381191481623, 37.752918829905944],
     [-122.38118397672194, 37.75291927864523],
     [-122.38117648352042, 37.75291983083678],
     [-122.38116900204152, 37.752920487381495],
     [-122.3811615379353, 37.75292124728795],
     [-122.38115409006679, 37.752922110574225],
     [-122.38114666295203, 37.752923076267074],
     [-122.38113925774793, 37.7529241452493],
     [-122.38113187783544, 37.752925316565594],
     [-122.38112452434947, 37.752926590198],
     [-122.38111720067063, 37.75292796519121],
     [-122.38110990793352, 37.752929441527215],
     [-122.38110027416761

In [55]:
output_filename = 'robbery-street-nhood.js'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(neighborhoods_js, output_file, indent=4) 